#Group Structure in Code

##Helper Functions

###Find the prime factors of the group's order to construct groups from its simple subgroups

In [1]:
def primeFactorization(number):
  primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
  if number in primes:
    return [number]
  prime_factors = []
  for prime in primes:
    if number == 1:
      return prime_factors
    while number % prime == 0:
      prime_factors.append(prime)
      number = number / prime
  return prime_factors

###Get the unique values of a list by testing if elements are equal, including overriden \_\_eq__ method

In [2]:
def unique(allElements):
  results = []
  for element in allElements:
    if element not in results:
      results.append(element)
  return results

###Calculates the mathematical factorial and choose operations

In [3]:
def factorial(n):
  if n==0:
    return 1
  else:
    return n*factorial(n-1)  

In [4]:
def choose(group, choice):
  return factorial(group) / (factorial(choice)*factorial(group-choice))

###Given a number, calculates all possible combinations of 3 indices without replacement

In [5]:
def getChooseThree(num):
  possibilities = []
  for i in range(num):
    for k in range(i+1, num):
      for j in range(k+1, num):
        possibilities.append([i, k, j])
  return possibilities

###Combines a list of lists into a single list

In [6]:
def listSum(listOfLists):
  result = listOfLists[0]
  for currList in listOfLists[1:]:
    result += currList
  return result

##Group Data Structure

###Given the Cayley Multiplication Table for a Group, tests for the conditions of closure, identity, associativity, and inverse

In [7]:
import numpy as np
import itertools
def verifyTable(table):
  table = np.array(table)
  if table.shape[0] != table.shape[1]:
    print("not square")
    return False
  for row in table:
    if len(np.unique(row)) != table.shape[0]:
      print("row is not unique")
      return False
  for column in table.T:
    if len(np.unique(column)) != table.shape[0]:
      print("Column is not unique")
      return False
  possibilities = getChooseThree(table.shape[0])
  for possibility in possibilities:
    for permutation in itertools.permutations(possibility):
      if table[permutation[0], table[permutation[1], permutation[2]]] != table[table[permutation[0],permutation[1]], permutation[2]]:
        print("Not Associative")
        return False
  return True

In [8]:
validZ4 = [
           [0,1,2,3],
           [1,2,3,0],
           [2,3,0,1],
           [3,0,1,2],
]
invalidZ4 = [
           [0,1,3,2],
           [1,2,0,3],
           [3,0,2,1],
           [2,3,1,0],
]
print(f"Valid: {verifyTable(validZ4)}")
print(f"Invalid: {verifyTable(invalidZ4)}")

Valid: True
Not Associative
Invalid: False


###A group can be created given its multiplication table, elements, and representation. It also has the property abelian which corresponds to whether or not the group operation is commutitive and the following methods:


*   checkAbelian -> tests if the group operation is commutitive
*   combine -> returns the result of the binary operation on two given elements in the group
*   combineMultiple -> combines all elements in the list from left to right
*   inverse -> finds the inverse of a given element in the group
*   order -> calculates the order of a given element in the group
*   \_\_eq__ -> determines if a group is isomorphic to another given group
*   \_\_repr__ -> Returns the group's representation in standard group notation
*   \_\_str__ -> Returns the group's multiplication table as a string









In [9]:
class Group:
  def __init__(self, table, newElements, repr):
    if not verifyTable(table):
      raise ValueError("Invalid Group")
    else:
      self.elements = newElements
      self.multiplication = table
      self.representation = repr
      self.abelian = self.checkAbelian()

  def checkAbelian(self):
    for i in range(len(self.elements)):
      for j in range(len(self.elements)):
        if self.multiplication[i,j] != self.multiplication[j,i]:
          return False
    return True
 
  def combine(self, element1, element2):
    try:
      return self.elements[self.multiplication[self.elements.index(element1), self.elements.index(element2)]]
    except:
      raise ValueError("Element not in in group")

  def combineMultiple(self, toCombine):
    currElement = toCombine[0]
    for element in toCombine[1:]:
      currElement = self.combine(currElement, element)
    return currElement

  def inverse(self, element):
    try:
      return self.elements[list(self.multiplication[self.elements.index(element)]).index(0)]
    except:
      raise ValueError("Element not in in group")

  def order(self, element):
    if self.combine(element, element) == element:
      return 0
    order = 2
    currElement = self.combine(element, element)
    while currElement != element:
      order += 1
      currElement = self.combine(currElement, element)
    return order - 1

  def __eq__(self, other):
    return sorted([self.order(element) for element in self.elements]) == sorted([other.order(element2) for element2 in other.elements]) and self.abelian==other.abelian
  
  def __repr__(self):
    return self.representation

  def __str__(self):
    return str(self.multiplication)

In [10]:
Z4 = Group(np.array(validZ4), [0, 1, 2, 3], 'Z4')
print(f"Elements: {Z4.elements}")
print(f"2+3 mod 4 = {Z4.combine(2,3)}") # 5 mod 4 is 1
print(f"Inverse of 3 mod 4: {Z4.inverse(3)}") # 3 + 1 mod 4 is 0
print(f"Order of 1 in Z4: {Z4.order(1)}") # 1 * 4 is 0 mod 4

Elements: [0, 1, 2, 3]
2+3 mod 4 = 1
Inverse of 3 mod 4: 1
Order of 1 in Z4: 4


##Generating Abelian Groups

### Returns the cyclic group of a given order n
Elements: [0,n-1]

Operation: Addition mod n

In [11]:
def cyclic(numElements):
  results = np.zeros((numElements, numElements), dtype=int)
  for i in range(numElements):
    for j in range(numElements):
      results[i, j] = int((i+j) % numElements)
  return Group(results, list(range(numElements)), "Z"+str(numElements))

In [12]:
print(f"Does the automatically generated group equal the manually generated one?: {cyclic(4) == Z4}")

Does the automatically generated group equal the manually generated one?: True


###Returns the direct product of two groups A and B
Elements: (a,b) where {a ∈ A, b ∈ B}

Operation: (a1,b1) + (a2,b2) -> (a1+a2,b1xb2) where + is operation of A and x is operation of B

In [13]:
from itertools import product
def cross(group1, group2):
  initialBounds = sorted([list(element) for element in list(product(group1.elements, group2.elements))])
  results = np.zeros((len(initialBounds), len(initialBounds)), dtype=int)
  for rowIndex, row in enumerate(initialBounds):
    for colIndex, col in enumerate(initialBounds):
      results[rowIndex,colIndex] = initialBounds.index([group1.combine(row[0], col[0]), group2.combine(row[1], col[1])])
  return Group(results, initialBounds, group1.representation + " X " + group2.representation)

In [14]:
Z2XZ2 = cross(cyclic(2), cyclic(2))
print(f"Elements: {Z2XZ2.elements}")
print(f"Binary sum without carrying of 01 and 11 = {Z2XZ2.combine([0,1],[1,1])}") # 10
print(f"Inverse of 01: {Z2XZ2.inverse([0,1])}") # Each element is its own inverse
print(f"Order of 01: {Z2XZ2.order([0,1])}") # Each element also has order 2

Elements: [[0, 0], [0, 1], [1, 0], [1, 1]]
Binary sum without carrying of 01 and 11 = [1, 0]
Inverse of 01: [0, 1]
Order of 01: 2


### Returns the direct product of all given groups from left to right

In [15]:
def crossMultiple(*groups):
  results = cross(groups[0], groups[1])
  for group in groups[2:]:
    results = cross(results, group)
  return results

##Generating Non-Abelian Groups

### (deprecated) Finds the automorphism group of a cyclic group

In [16]:
def generateAutos(order):
  possible_autos = {1:0}
  for i in range(2,order):
    elementOrder = 1
    currentNum = i
    while currentNum % order != 1:
      elementOrder += 1
      currentNum *= i
    possible_autos[i] = elementOrder
  return possible_autos

###Determines if a given set of elements generates a given Group

In [17]:
def generates(group, elements):
  generatedElements = elements.copy() 
  order = len(group.elements)
  for i in range(order):
    for element1 in elements:
      for element2 in elements:
        if group.combine(element1, element2) not in generatedElements:
          generatedElements.append(group.combine(element1, element2))
    elements = generatedElements.copy()
  allThere = True
  for element in group.elements:
    if element not in generatedElements:
      allThere = False
      break
  return allThere

In [18]:
print(f"Can one element generate Z2 X Z2: {generates(cross(cyclic(2), cyclic(2)), [[0,1]])}") #Z2 X Z2 is not cyclic
print(f"Can one element generate Z4: {generates(cyclic(4), [1])}") #Z4 is cyclic

Can one element generate Z2 X Z2: False
Can one element generate Z4: True


###Finds all of the smallest possible sets of elements that generate a given group

In [19]:
def generators(group):
  generators = group.elements.copy()[1:]
  for i in range(len(group.elements)):
    removed = True
    while removed:
      removed = False
      index = 0
      while index < len(generators):
        if generates(group, generators[:index]+generators[index+1:]):
          generators.pop(index)
          removed = True
        else:
          index += 1
  basis_length = len(generators)
  actual_bases = []
  all_elements = group.elements.copy()[1:]
  counters = list(range(basis_length))
  for k in range(int(round(choose(len(all_elements), basis_length)))):
    test_basis = [all_elements[counter] for counter in counters]
    if generates(group, test_basis):
      actual_bases.append(test_basis)
    for j in range(len(counters)):
      if counters[-(j+1)] < len(all_elements)-1:
        counters[-(j+1)] += 1
        break
  return actual_bases

In [20]:
print(f"Generators of Z2 X Z2: {generators(cross(cyclic(2), cyclic(2)))}") #Z2 X Z2 basis vectors are length 2
print(f"Generators of Z4: {generators(cyclic(4))}") #Z4 has two single generators

Generators of Z2 X Z2: [[[0, 1], [1, 0]], [[0, 1], [1, 1]], [[1, 0], [1, 1]]]
Generators of Z4: [[1], [3]]


###Given an automorphism as [preimage,image], returns a function which determines where an element is mapped in the automorphism

In [21]:
def auto_function(auto_list):
  def phism(element):
    return auto_list[1][auto_list[0].index(element)]
  return phism

###Given two possible sets of generators of a group, finds the automorphism created by mapping the first set to the second

In [22]:
def automorphism(group, basis1, basis2):
  foundElements = basis1.copy()
  foundMappings = basis2.copy()
  testedCoefficients = [1] + [0] * (len(basis1) - 1)
  while len(foundElements) != len(group.elements):
    currElement = group.combineMultiple(listSum([testedCoefficients[i]*[basis1[i]] for i in range(len(basis1))])) 
    currMap = group.combineMultiple(listSum([testedCoefficients[i]*[basis2[i]] for i in range(len(basis1))]))
    if currElement not in foundElements:
      foundElements.append(currElement)
      foundMappings.append(currMap)
    for idx in range(len(testedCoefficients)):
      coef = testedCoefficients[-idx-1]
      if coef < len(group.elements) + 1:
        testedCoefficients[-idx-1] = coef+1
        break
      else:
        testedCoefficients[-idx-1] = 0
  twobytwo = list(sorted(zip(foundElements, foundMappings), key=lambda x:x[0]))
  return [[element[0] for element in twobytwo],[element2[1] for element2 in twobytwo]]

###Returns the automorphism group of a given group G

Elements: All possible automorphisms of G

Operation: The automorphisms which results when two given automorphisms are applied one after the other

In [23]:
def generalizedAutos(group):
  bases = generators(group)
  autos = []
  for basis1 in bases:
    for basis2 in bases[1:]:
      newAuto = automorphism(group, basis1, basis2)
      if newAuto not in autos:
        autos.append(newAuto)
  for idx, auto in enumerate(autos):
    if auto[0] == auto[1]:
      autos.insert(0, autos.pop(idx))
  for row in autos:
    for col in autos:
      domain = row[0]
      image = [col[1][col[0].index(firstMap)] for firstMap in row[1]]
      if [domain,image] not in autos:
        autos.append([domain,image])
  results = np.zeros((len(autos), len(autos)), dtype=int)
  for rowIndex, row in enumerate(autos):
    for colIndex, col in enumerate(autos):
      domain = row[0]
      image = [col[1][col[0].index(firstMap)] for firstMap in row[1]]
      results[rowIndex, colIndex] = autos.index([domain, image])
  return Group(results, autos, f"Aut({group.representation})")


In [24]:
Z4autos = generalizedAutos(cyclic(4))
Z2XZ2autos = generalizedAutos(cross(cyclic(2), cyclic(2)))
print(f"Z4 Identity Aut of 1: {auto_function(Z4autos.elements[0])(1)}")
print(f"Z2XZ2 Non Trivial Aut of [0,1]: {auto_function(Z2XZ2autos.elements[2])([0,1])}")
newAuto = Z2XZ2autos.combine(Z2XZ2autos.elements[2], Z2XZ2autos.elements[3])
print(auto_function(newAuto)([0,1]))

Z4 Identity Aut of 1: 1
Z2XZ2 Non Trivial Aut of [0,1]: [1, 0]
[0, 1]


###Given two groups G and H, calculates all homomorphisms between the two groups

Finds all possible mappings F between G and H, and for each mapping f ∈ F and each element pair g1,g2 ∈ G tests if f(g1*g2) = f(g1) * f(g2)

In [25]:
def homomorphism(group1, group2):
  group1generators = group1.elements.copy()
  group2orders = [group2.order(element) for element in group2.elements]
  homos = []
  possibleMappings = []
  for generator in group1generators:
    possibleMappings.append([group2.elements[idx] for idx,val in enumerate(group2orders)])
  mapIndices = [0] * len(possibleMappings)
  finished=False
  while not finished:
    currentMap = [possibleMappings[index][mapIndex] for index, mapIndex in enumerate(mapIndices)]
    testResults = np.zeros((len(currentMap), len(currentMap)), dtype=int)
    homo=True
    for element1, map1 in zip(group1.elements,currentMap):
      if not homo:
        break
      for element2, map2 in zip(group1.elements,currentMap):
        if currentMap[group1.elements.index(group1.combine(element1, element2))] != group2.combine(currentMap[group1.elements.index(element1)], currentMap[group1.elements.index(element2)]):
          homo=False
          break
    if homo:
      homos.append(currentMap)
    for idx,val in enumerate(mapIndices):
      if val < len(possibleMappings[idx]) - 1:
        mapIndices[idx] += 1
        break
      finished = (idx == len(mapIndices) - 1)
      mapIndices[idx] = 0
  return homos

###(deprecated) Finds the semidirect product of two cyclic groups of given prime orders where prime2 mod prime1 is 1

In [26]:
def generateNonAbelian(prime1, prime2):
  if prime1>prime2:
    prime1,prime2 = prime2,prime1
  initialBounds = list(product(range(prime1), range(prime2)))
  for idx, val in enumerate(initialBounds):
    try:
      temp = list(val[0])
    except:
      temp = [val[0]]
    temp.append(val[1])
    initialBounds[idx] = temp
  results = np.zeros((len(initialBounds), len(initialBounds)), dtype=int)
  groupH = cyclic(prime1)
  groupN = cyclic(prime2)
  autoHomomorphism = [phism[0] for phism in generateAutos(prime2).items() if phism[1] in [0, prime1]]
  for rowIndex, row in enumerate(initialBounds):
    for colIndex, col in enumerate(initialBounds):
      results[rowIndex, colIndex] = initialBounds.index([groupH.combine(row[0],col[0]), groupN.combine((row[1] * autoHomomorphism[groupH.inverse(col[0])]) % prime2, col[1])])
  return Group(results, initialBounds, f"Z{prime1} ⋊ Z{prime2}")

### Calculates the semidirect product of two given groups A and B using a given homomorphism ϕ from the first group to the automorphism group of the second group

Elements: (a,b) where {a ∈ A, b ∈ B}

Operation: (a1,b1) + (a2,b2) -> (a1+a2,ϕ(a2<sup>-1</sup>)(b1)xb2) where + is operation of A and x is operation of B

In [27]:
def semiDirectProduct(group1, group2, homomorphism):
  initialBounds = sorted([list(element) for element in list(product(group1.elements, group2.elements))])
  results = np.zeros((len(initialBounds), len(initialBounds)), dtype=int)
  for rowIndex, row in enumerate(initialBounds):
    for colIndex, col in enumerate(initialBounds):
      secondElement = auto_function(homomorphism[group1.elements.index(group1.inverse(col[0]))])(row[1])
      results[rowIndex,colIndex] = initialBounds.index([group1.combine(row[0], col[0]), group2.combine(secondElement, col[1])])
  return Group(results, initialBounds, group1.representation + " ⋊ " + group2.representation)

###Finds all possible semidirect products between two given groups and the given remaining factor when the target order is divided by the product of the two groups' orders 

In [28]:
def allSemiDirects(group1, group2, remainder):
  results = []
  if len(group2.elements) == 2:
    return results
  possibleHomos = homomorphism(group1, generalizedAutos(group2))
  for homo in possibleHomos:
    semiDirect = semiDirectProduct(group1, group2, homo)
    if remainder==1:
      results.append(semiDirect)
    else:
      results.append(cross(semiDirect, cyclic(remainder)))
  return results

##Calulates almost all groups up to isomorphism of given order n including abelian, non-abelian, cyclic, non-cyclic, etc. Becomes unreasonable to do automatically when n > 20, but it is very reasonable to do so using the cyclic, cross, and semidirect functions, which can be used to represent almost any group.

In [29]:
def generateAll(numElements):
  factors = primeFactorization(numElements)
  results = [cyclic(numElements)]
  uniqueFactors = list(np.unique(factors))
  for index, factor in enumerate(uniqueFactors):
    if factors.count(factor) >= 2:
      remainder = int(round(numElements / factor**2))
      if remainder!=1:
        results.append(crossMultiple(cyclic(factor), cyclic(factor), cyclic(remainder)))
      else:
        results.append(cross(cyclic(factor), cyclic(factor)))
    for secondFactor in uniqueFactors[index+1:]:
      if secondFactor % factor == 1:
        for i in range(factors.count(factor)):
          for j in range(factors.count(secondFactor)):
            cyclicRemainder = int(round(numElements / (factor**(i+1) * secondFactor**(j+1))))
            results = results + allSemiDirects(cyclic(factor**(i+1)), cyclic(secondFactor**(j+1)), cyclicRemainder)
            currSmall = cyclic(factor)
            currBig = cyclic(secondFactor)
            for numSmallCrosses in range(i):
              currSmall = cross(currSmall, cyclic(factor))
            for numBigCrosses in range(j):
              currBig = cross(currSmall, cyclic(factor))
            remainder = int(round(numElements / (factor**(i+1) * secondFactor**(j+1))))
            results = results + allSemiDirects(currSmall, currBig, remainder)
            results = results + allSemiDirects(currBig, currSmall, remainder)
  return unique(results)

In [30]:
generateAll(12)

[Z12, Z2 X Z2 X Z3, Z2 ⋊ Z3 X Z2, Z4 ⋊ Z3, Z3 ⋊ Z2 X Z2]